In [1]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.python.keras.datasets import mnist
from tensorflow.contrib.eager.python import tfe

D:\Users\Yue\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# enable eager mode
tf.enable_eager_execution()
tf.set_random_seed(0)
np.random.seed(0)

In [3]:
# constants
batch_size = 128
epochs = 8
num_classes = 10

In [4]:
# dataset loading
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

# one hot encode the labels. convert back to numpy as we cannot use a combination of numpy
# and tensors as input to keras
y_train_ohe = tf.one_hot(y_train, depth=num_classes).numpy()
y_test_ohe = tf.one_hot(y_test, depth=num_classes).numpy()

print('x train', x_train.shape)
print('y train', y_train_ohe.shape)
print('x test', x_test.shape)
print('y test', y_test_ohe.shape)

x train (60000, 28, 28, 1)
y train (60000, 10)
x test (10000, 28, 28, 1)
y test (10000, 10)


# Convolutional Networks
A basic conv net with 2 `Conv-BatchNorm-Relu` blocks. This is the canonical way to define a small network.

However, we can also decompose the blocks themselves into another class which also extends Model, and then use objects of that Model inside another Model. This is allowed, and follows OOP principles, so I will be using this pattern from now on.

The example of Model as a layer can be seen in `04_02_cnn.ipynb`

In [5]:
# model definition (canonical way)
class CNN(tf.keras.Model):

    def __init__(self, num_classes):
        super(CNN, self).__init__()

        self.cnn1 = tf.keras.layers.Conv2D(16, (5, 5), padding='same', strides=(2, 2),
                                           kernel_initializer='he_normal')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.cnn2 = tf.keras.layers.Conv2D(32, (5, 5), padding='same', strides=(2, 2),
                                           kernel_initializer='he_normal')
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.pool = tf.keras.layers.GlobalAveragePooling2D()
        self.classifier = tf.keras.layers.Dense(num_classes)

    def call(self, inputs, training=None, mask=None):
        x = self.cnn1(inputs)
        x = self.bn1(x)
        x = tf.nn.relu(x)  # layer 1
        x = tf.nn.relu(self.bn2(self.cnn2(x)))  # layer 2
        x = self.pool(x)
        output = self.classifier(x)

        # softmax op does not exist on the gpu, so always use cpu
        with tf.device('/cpu:0'):
            output = tf.nn.softmax(output)

        return output

In [6]:
device = '/cpu:0' if tfe.num_gpus() == 0 else '/gpu:0'

with tf.device(device):
    # build model and optimizer
    model = CNN(num_classes)
    model.compile(optimizer=tf.train.AdamOptimizer(0.001), loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # TF Keras tries to use entire dataset to determine shape without this step when using .fit()
    # Fix = Use exactly one sample from the provided input dataset to determine input/output shape/s for the model
    dummy_x = np.zeros((1, 28, 28, 1))
    model._set_inputs(dummy_x)

    # train
    model.fit(x_train, y_train_ohe, batch_size=batch_size, epochs=epochs,
              validation_data=(x_test, y_test_ohe), verbose=1)

    # evaluate on test set
    scores = model.evaluate(x_test, y_test_ohe, batch_size, verbose=1)
    print("Final test loss and accuracy :", scores)

Train on 60000 samples, validate on 10000 samples
Epoch 1/8
60000/60000 [==============================] - 13s 212us/step - loss: 1.3410 - acc: 0.6647 - val_loss: 0.9821 - val_acc: 0.7298
Epoch 2/8
60000/60000 [==============================] - 11s 186us/step - loss: 0.5720 - acc: 0.8861 - val_loss: 0.5948 - val_acc: 0.8021
Epoch 3/8
60000/60000 [==============================] - 11s 177us/step - loss: 0.3462 - acc: 0.9251 - val_loss: 0.4176 - val_acc: 0.8774
Epoch 4/8
60000/60000 [==============================] - 11s 177us/step - loss: 0.2550 - acc: 0.9419 - val_loss: 0.3381 - val_acc: 0.9110
Epoch 5/8
60000/60000 [==============================] - 11s 175us/step - loss: 0.2048 - acc: 0.9513 - val_loss: 0.2127 - val_acc: 0.9420
Epoch 6/8
60000/60000 [==============================] - 10s 167us/step - loss: 0.1732 - acc: 0.9580 - val_loss: 0.2316 - val_acc: 0.9397
Epoch 7/8
60000/60000 [==============================] - 11s 184us/step - loss: 0.1517 - acc: 0.9626 - val_loss: 0.1862 - 